In [1]:
import numpy as np
import pandas as pd
import requests
import json

A) Use el API de institutions para extraer la lista de instituciones del país, por ejemplo: colombia

In [3]:
countries = json.load(open('data_json.json'))
random_country = np.random.choice(countries)['Name']
random_country

'Moldova, Republic of'

In [4]:
url = "https://inspirehep.net/api/institutions?q={}".format(random_country)
institution_link = requests.get(url)

legacy_ICN = [institution.get('metadata').get('legacy_ICN') if institution.get('metadata').get('number_of_papers')>0 else None for institution in institution_link.json().get('hits').get('hits')]

legacy_ICN

['IAP, Chisinau']

B) Con el API de literatura obtenga el JSON con los artículos de menos de 10 autores usando el "legacy_ICN" de la siguiente manera

In [5]:
literature_link ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(legacy_ICN[0].replace(' ','+'))
literature_link = requests.get(literature_link)

articles = [article.get('metadata') for article in literature_link.json().get('hits').get('hits')]
n=len(articles)

C) Para al menos un artículo de esa institución, extraiga el URL del perfil de cada autor de esa institución que se encuentra dentro del campo "authors" en "record" y luego en "$ref"

In [6]:
authors_link = [author.get('record').get('$ref') for author in articles[np.random.randint(0,n)].get('authors')]
authors_link

['https://inspirehep.net/api/authors/1057005',
 'https://inspirehep.net/api/authors/1002323',
 'https://inspirehep.net/api/authors/1049032']

D) Con cada uno de los datos del resultado del API para cada perfil construya una tabla con los siguientes columnas (puede que alguno de los datos no esté disponible): 

In [7]:
INF = [requests.get(i).json().get('metadata') for i in authors_link]

In [8]:
INF

[{'positions': [{'record': {'$ref': 'https://inspirehep.net/api/institutions/902780'},
    'current': True,
    'institution': 'Dubna, JINR',
    'curated_relation': True}],
  'ids': [{'value': 'A.S.Khvorostukhin.1', 'schema': 'INSPIRE BAI'},
   {'value': 'HEPNAMES-831662', 'schema': 'SPIRES'}],
  'name': {'value': 'Khvorostukhin, A.S.',
   'preferred_name': 'A.S. Khvorostukhin'},
  'stub': False,
  'status': 'active',
  '$schema': 'https://inspirehep.net/schemas/records/authors.json',
  'deleted': False,
  'control_number': 1057005,
  'legacy_version': '20150923144708.0',
  'arxiv_categories': ['nucl-th'],
  'legacy_creation_date': '2008-06-30'},
 {'advisors': [{'ids': [{'value': 'INSPIRE-00094246', 'schema': 'INSPIRE ID'}],
    'name': 'Soff, Gerhard',
    'record': {'$ref': 'https://inspirehep.net/api/authors/1003956'},
    'degree_type': 'other'},
   {'ids': [{'value': 'INSPIRE-00193968', 'schema': 'INSPIRE ID'}],
    'name': 'Kampfer, Burkhard',
    'record': {'$ref': 'https://ins

In [9]:
def f(x):
  try:
    return x.get('name').get('value')
  except: 
    return None

In [10]:
name = list(map(f,INF))

In [23]:
def f(x):
  try:
    return x.get('email_addresses')[0].get('value')
  except: 
    return None

In [26]:
email = list(map(f,INF))

In [27]:
def f(x):
  try:
    return x.get('positions')[0].get('rank')
  except: 
    return None

In [28]:
position = list(map(f,INF))

In [29]:
def f(x):
  try:
    return x.get('positions')[0].get('start_date') 
  except: 
    return None

In [30]:
sdate = list(map(f,INF))

In [31]:
def f(x):
  try:
    return x.get('positions')[0].get('end_date') 
  except: 
    return None

In [32]:
edate = list(map(f,INF))

In [44]:
def f(x):
  try:
    return x.get('positions')[0].get('institution') 
  except: 
    return None

In [45]:
institution = list(map(f,INF))

In [46]:
df = pd.DataFrame({'NAME ': name ,
        'e-mail': email,
        'POSITIONS': position,
        'INSTITUTION': institution,
        'START DATE': sdate,
        'END DATE': edate})

df

,NAME,e-mail,POSITIONS,INSTITUTION,START DATE,END DATE
0,"Khvorostukhin, A.S.",None,None,"Dubna, JINR",None,None
1,"Kolomeitsev, Evgeni E.",e.kolomeitsev@gsi.de,POSTDOC,"Darmstadt, GSI",None,None
2,"Toneev, Viacheslav D.",toneev@clri6a.gsi.de,None,"Darmstadt, GSI",None,None
